In [ ]:
import tensorflow as tf

# Batch Queue를 이용, 여러 Data 소스를 불러오기
# 여러 방법들은 공식문서 참조
filename_queue = tf.train.string_input_producer(
    ["data1.csv", "data2.csv", "data3.csv" ],
    shuffle=False, name = "filename_queue"
)

# 불러온 소스에 대한 읽기 처리
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

In [ ]:
record_defaults = [[0.], [0.], [0.], [0.]]
# Data에 대한 decoding 처리
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [ ]:
# node의 이름 정의, 각 Data 위치, 요청 시 몇 개씩 가져올지
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size = 10)

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [ ]:
# Shape 항상 유의
W = tf.Variable(tf.random_normal([3, 1]), name = "weight")
b = tf.Variable(tf.random_normal([1]), name = "bias")

In [ ]:
hypo = tf.matmul(X, W) + b

In [ ]:
cost = tf.reduce_mean(tf.square(hypo - Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
#Batch Queue를 위한 시작점
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [ ]:
# 학습 값은 batch Data
for step in range(2001) : 
    x_batch , y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypo, train], feed_dict={X:x_batch, Y:y_batch}
    )
    if step % 10 == 0 :
        print(step, "Cost : ", cost_val, "Prediction : ", hy_val)

In [ ]:
#Batch Queue를 위한 끝 점
coord.request_stop()
coord.join(threads)